In [11]:
$src = "D:\Development\SDL\include";
$dst = ".\src\SharpSDL\Interop";
$log = "$dst\genbinds_log.txt";

$headers = 
"SDL_audio.h ",
"SDL_blendmode.h",
"SDL_clipboard.h",
"SDL_cpuinfo.h",
"SDL_error.h",
"SDL_events.h",
"SDL_filesystem.h",
"SDL_gamecontroller.h",
"SDL_gesture.h",
"SDL_guid.h",
"SDL_haptic.h",
"SDL_hints.h",
"SDL_joystick.h",
"SDL_keyboard.h",
"SDL_keycode.h",
"SDL_locale.h",
"SDL_log.h",
"SDL_messagebox.h",
"SDL_metal.h",
"SDL_misc.h",
"SDL_mouse.h",
"SDL_pixels.h",
"SDL_platform.h",
"SDL_power.h",
"SDL_rect.h",
"SDL_render.h",
"SDL_revision.h",
"SDL_rwops.h",
"SDL_scancode.h",
"SDL_sensor.h",
"SDL_shape.h",
"SDL_stdinc.h",
"SDL_surface.h",
"SDL_system.h",
"SDL_syswm.h",
"SDL_timer.h",
"SDL_touch.h",
"SDL_version.h",
"SDL_video.h",
"SDL_vulkan.h",
"SDL.h";

[System.IO.File]::Delete($log);

Get-ChildItem $dst -Filter "SDL.*" | ForEach-Object {
    [System.IO.File]::Delete($_);
}

$headers | ForEach-Object {
    $in = "$src\$($_)";
    $out = "$dst\$([System.IO.Path]::ChangeExtension($_.Replace("SDL_", "SDL."), ".cs"))";
    ClangSharpPInvokeGenerator `
        --file $in `
        --output $out `
        --language "c" `
        --libraryPath "SDL2" `
        --namespace "SharpSDL.Interop" `
        --methodClassName "SDL2" `
        --prefixStrip "SDL_" `
        --exclude "SDL_bool" `
        --exclude "DUMMY_ENUM" `
        --nativeTypeNamesToStrip `
        "Uint8" `
        "const Uint8" `
        "Uint8 *" `
        "const Uint8 *" `
        "Uint16" `
        "const Uint16" `
        "Uint16 *" `
        "const Uint16 *" `
        "Uint32" `
        "const Uint32" `
        "Uint32 *" `
        "const Uint32 *" `
        "Uint64" `
        "const Uint64" `
        "Uint64 *" `
        "const Uint64 *" `
        "Sint8" `
        "const Sint8" `
        "Sint8 *" `
        "const Sint8 *" `
        "Sint16" `
        "const Sint16" `
        "Sint16 *" `
        "const Sint16 *" `
        "Sint32" `
        "const Sint32" `
        "Sint32 *" `
        "const Sint32 *" `
        "Sint64" `
        "const Sint64" `
        "Sint64 *" `
        "const Sint64 *" `
        --remap "sbyte=byte" `
        --remap "_SDL_AudioStream*=nint" `
        --remap "_SDL_GameController*=nint" `
        --remap "_SDL_Haptic*=nint" `
        --remap "_SDL_iconv_t*=nint" `
        --remap "_SDL_Joystick*=nint" `
        --remap "_SDL_Sensor*=nint" `
        --remap "HDC__*=nint" `
        --remap "HINSTANCE__*=nint" `
        --remap "HWND__*=nint" `
        --remap "IDirect3DDevice9*=nint" `
        --remap "ID3D11Device*=nint" `
        --remap "ID3D12Device*=nint" `
        --remap "SDL_bool=CBool" `
        --remap "SDL_BlitMap*=nint" `
        --remap "SDL_Cursor*=nint" `
        --remap "SDL_Renderer*=nint" `
        --remap "SDL_Texture*=nint" `
        --remap "SDL_Window*=nint" `
        --remap "VkInstance_T*=nint" `
        --remap "VkSurfaceKHR_T*=nint" `
        --config preview-codegen `
        --config generate-macro-bindings `
        --config generate-unmanaged-constants `
        --config exclude-using-statics-for-enums `
        --config exclude-funcs-with-body `
        --config exclude-empty-records `
    | Out-File $log -Append
    #--config generate-file-scoped-namespaces `
    #--config generate-helper-types `
    #--config log-visited-files `
}

$dst

.\src\SharpSDL\Interop


In [7]:
#r "nuget: Microsoft.CodeAnalysis.CSharp, 4.9.2"
#nullable enable
using Microsoft.CodeAnalysis;
using Microsoft.CodeAnalysis.CSharp;
using Microsoft.CodeAnalysis.CSharp.Syntax;

public sealed class BindingsRewriter : CSharpSyntaxRewriter
{
#nullable enable
    internal static SyntaxTrivia NewLine => Environment.NewLine == "\r\n" ? SyntaxFactory.CarriageReturnLineFeed : SyntaxFactory.LineFeed;

    private static readonly HashSet<string> ConstantsToRemove = [
        "HAVE_WINAPIFAMILY_H",
        "WINAPI_FAMILY_WINRT",
        "SDL_WINAPI_FAMILY_PHONE",
        "__WINDOWS__",
        "__WIN32__",
    ];

    public override SyntaxNode? VisitFieldDeclaration(FieldDeclarationSyntax node)
    {
        if (node.Declaration.Variables.Any(v => ConstantsToRemove.Contains(v.Identifier.Text)))
            return null;

        return base.VisitFieldDeclaration(node);
    }

    public override SyntaxNode? VisitEnumDeclaration(EnumDeclarationSyntax node)
    {
        if (node.Identifier.Text == "SDL_PixelFormatEnum")
        {
            node = node
                .WithIdentifier(node.Identifier.WithTrailingTrivia(SyntaxFactory.Space))
                .WithBaseList(
                    SyntaxFactory.BaseList(
                        SyntaxFactory.SingletonSeparatedList<BaseTypeSyntax>(
                            SyntaxFactory.SimpleBaseType(
                                SyntaxFactory.ParseTypeName("uint")
                                    .WithLeadingTrivia(SyntaxFactory.Space)
                                    .WithTrailingTrivia(NewLine)))));
        }
        return base.VisitEnumDeclaration(ReplacePublicWithInternalModifier(node));
    }

    public override SyntaxNode? VisitStructDeclaration(StructDeclarationSyntax node)
    {
        return base.VisitStructDeclaration(ReplacePublicWithInternalModifier(node));
    }

    public override SyntaxNode? VisitClassDeclaration(ClassDeclarationSyntax node)
    {
        return base.VisitClassDeclaration(ReplacePublicWithInternalModifier(node));
    }

    public override SyntaxNode? VisitMethodDeclaration(MethodDeclarationSyntax node)
    {
        node = ReplaceDllImportWithLibraryImport(node)!;
        if (node is null)
            return null;
        return base.VisitMethodDeclaration(ReplaceExternWithPartialModifier(node));
    }

    public override SyntaxNode? VisitParameterList(ParameterListSyntax node)
    {
        var argList = node.Parameters.SingleOrDefault(p => p.Identifier.Text == "__arglist" && p.Type is null);
        if (argList is not null)
            node = SyntaxFactory.ParameterList(node.Parameters.Remove(argList));
        return base.VisitParameterList(node);
    }

    private static T ReplacePublicWithInternalModifier<T>(T node) where T : MemberDeclarationSyntax
    {
        if (node.Modifiers.Any(d => d.IsKind(SyntaxKind.PublicKeyword)))
        {
            var publicKeyword = node.Modifiers.Single(d => d.IsKind(SyntaxKind.PublicKeyword));
            var internalKeyword = SyntaxFactory.Token(publicKeyword.LeadingTrivia, SyntaxKind.InternalKeyword, publicKeyword.TrailingTrivia);
            var modifiers = node.Modifiers.Replace(publicKeyword, internalKeyword);
            return (T)node.WithModifiers(modifiers);
        }
        return node;
    }

    private static MethodDeclarationSyntax ReplaceExternWithPartialModifier(MethodDeclarationSyntax node)
    {
        var externKeyword = node.Modifiers.Single(d => d.IsKind(SyntaxKind.ExternKeyword));
        var partialKeyword = SyntaxFactory.Token(externKeyword.LeadingTrivia, SyntaxKind.PartialKeyword, externKeyword.TrailingTrivia);
        var modifiers = node.Modifiers.Replace(externKeyword, partialKeyword);
        return node.WithModifiers(modifiers);
    }

    private static MethodDeclarationSyntax? ReplaceDllImportWithLibraryImport(MethodDeclarationSyntax node)
    {
        var list = node.AttributeLists.SingleOrDefault(list => list.Attributes.SingleOrDefault(attr => attr.Name.ToString() == "DllImport") is not null);
        if (list is not null)
        {
            var attrLists = node.AttributeLists;

            var dllImport = list.Attributes.Single(attr => attr.Name.ToString() == "DllImport");

            var entryPoint = dllImport.ArgumentList?.Arguments.SingleOrDefault(a => a.NameEquals?.ToFullString() == "EntryPoint = ");

            if (entryPoint is null)
                return null;

            var libraryImport = SyntaxFactory.Attribute(
                GetIdentifierWithTrivia(dllImport, "LibraryImport"),
                SyntaxFactory.ParseAttributeArgumentList($"({dllImport.ArgumentList?.Arguments[0].ToFullString()}, {entryPoint})"));
            var libraryImportList = list
                .WithAttributes(list.Attributes.Replace(dllImport, libraryImport))
                .WithTrailingTrivia(NewLine);
            attrLists = attrLists.Replace(list, libraryImportList);

            var unmanagedCallConv = SyntaxFactory.Attribute(
                GetIdentifierWithTrivia(dllImport, "UnmanagedCallConv"),
                SyntaxFactory.ParseAttributeArgumentList("(CallConvs = [typeof(CallConvCdecl)])"));
            var unmanagedCallConvList = SyntaxFactory.AttributeList(SyntaxFactory.SingletonSeparatedList(unmanagedCallConv))
                .WithLeadingTrivia(SyntaxFactory.TriviaList(libraryImportList.GetLeadingTrivia().Where(t => !t.IsKind(SyntaxKind.EndOfLineTrivia))))
                .WithTrailingTrivia(NewLine);
            attrLists = attrLists.Insert(attrLists.IndexOf(list => list.Attributes.Any(a => a.Name.ToString() == "LibraryImport")) + 1, unmanagedCallConvList);

            node = node.WithAttributeLists(attrLists);

            static IdentifierNameSyntax GetIdentifierWithTrivia(AttributeSyntax attribute, string newName)
            {
                return SyntaxFactory.IdentifierName(
                    SyntaxFactory.Identifier(
                        attribute.GetLeadingTrivia(),
                        newName,
                        attribute.GetTrailingTrivia()));
            }
        }
        return node;
    }
#nullable restore
}

Installed Packages Microsoft.CodeAnalysis.CSharp, 4.9.2

In [8]:
#!set --value @pwsh:dst --name dst

using System.Collections.Generic;
using System.IO;
using System;
using System.Linq;

var files = Directory.GetFiles(dst, "SDL*.cs");

foreach(var file in files) {

    var tree = CSharpSyntaxTree.ParseText(File.ReadAllText(file));

    var root = new BindingsRewriter().Visit(tree.GetRoot().WithLeadingTrivia(SyntaxFactory.Comment($"// <auto-generated />{Environment.NewLine}")));

    var text = root.ToFullString();

    File.WriteAllText(file, text);
}

Error: System.InvalidOperationException: Sequence contains no matching element
   at System.Linq.ThrowHelper.ThrowNoMatchException()
   at System.Linq.Enumerable.Single[TSource](IEnumerable`1 source, Func`2 predicate)
   at Submission#3.BindingsRewriter.ReplaceDllImportWithLibraryImport(MethodDeclarationSyntax node)
   at Submission#3.BindingsRewriter.VisitMethodDeclaration(MethodDeclarationSyntax node)
   at Microsoft.CodeAnalysis.CSharp.Syntax.MethodDeclarationSyntax.Accept[TResult](CSharpSyntaxVisitor`1 visitor) in /_/src/Compilers/CSharp/Portable/CSharpSyntaxGenerator/CSharpSyntaxGenerator.SourceGenerator/Syntax.xml.Syntax.Generated.cs:line 12068
   at Microsoft.CodeAnalysis.CSharp.CSharpSyntaxRewriter.Visit(SyntaxNode node) in /_/src/Compilers/CSharp/Portable/Syntax/CSharpSyntaxRewriter.cs:line 41
   at Microsoft.CodeAnalysis.CSharp.CSharpSyntaxRewriter.VisitListElement[TNode](TNode node) in /_/src/Compilers/CSharp/Portable/Syntax/CSharpSyntaxRewriter.cs:line 168
   at Mic